# The main goal of this project is:
1. How much the senators spent with benefits paid for the governament
2. Found the company that have more sells for the senators
3. How much employee each senators have
4. How much cost the senators employee per year

# Star Schema
Fact table with the goals
Dims: Senators, Companies, EmployeesCabinet, Employees, Benefits

# Init Project

In [1]:
# imports
# Imports
import os

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Test Setup")\
        .getOrCreate()

22/11/02 15:05:37 WARN Utils: Your hostname, MacBook-Pro-de-Paulo-Bernardo.local resolves to a loopback address: 127.0.0.1; using 192.168.15.4 instead (on interface en0)
22/11/02 15:05:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/02 15:05:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# How much the senators spent with benefits paid for the governament

In [3]:
senators = spark.read.parquet("database/silver/senators")

In [4]:
benefits = spark.read.parquet("database/silver/benefits")

In [5]:
senators_benefits = senators.join(benefits, f.upper(f.col("NomeParlamentar")) == f.upper(f.col("SENADOR")), how="inner")

In [6]:
senators_benefits.show(n=1)

+----------+----------------+---------------+-----------------------+----+---+------------+--------------------+------------------+--------------------+---------+----------+------------+-----------------+-------------+
|row_number|codigoPalamentar|NomeParlamentar|NomeCompletoParlamentar| ANO|MES|     SENADOR|        TIPO_DESPESA|          CNPJ_CPF|          FORNECEDOR|DOCUMENTO|      DATA|DETALHAMENTO|VALOR_REEMBOLSADO|COD_DOCUMENTO|
+----------+----------------+---------------+-----------------------+----+---+------------+--------------------+------------------+--------------------+---------+----------+------------+-----------------+-------------+
|         0|            4981|   Acir Gurgacz|    Acir Marcos Gurgacz|2013|  1|ACIR GURGACZ|Aluguel de imóvei...|05.914.650/0001-66|CENTRAIS ELÉTRICA...|000077305|18/01/2013|        null|           214.79|       705442|
+----------+----------------+---------------+-----------------------+----+---+------------+--------------------+------------

In [7]:
senators_benefits.count()

101702

In [8]:
senators_spent_benefits = senators_benefits.select("NomeCompletoParlamentar", "VALOR_REEMBOLSADO", "ANO")\
                        .groupBy("NomeCompletoParlamentar", "ANO")\
                        .agg(f.sum("VALOR_REEMBOLSADO").alias("VALOR_REEMBOLSADO"))\
                        .orderBy("VALOR_REEMBOLSADO", ascending=False).show(truncate=False)

+--------------------------------+----+------------------+
|NomeCompletoParlamentar         |ANO |VALOR_REEMBOLSADO |
+--------------------------------+----+------------------+
|David Samuel Alcolumbre Tobelem |2020|513609.0          |
|David Samuel Alcolumbre Tobelem |2021|508046.74         |
|David Samuel Alcolumbre Tobelem |2016|472266.99000000005|
|Sérgio de Oliveira Cunha        |2014|464327.38000000006|
|Sérgio de Oliveira Cunha        |2013|464189.94999999995|
|David Samuel Alcolumbre Tobelem |2017|461327.22         |
|Sérgio de Oliveira Cunha        |2012|460001.0700000001 |
|Antônio Mecias Pereira de Jesus |2020|447756.17         |
|Telmário Mota de Oliveira       |2020|445444.94999999995|
|Jader Fontenelle Barbalho       |2013|441914.39999999997|
|Rogério Carvalho Santos         |2020|438867.0500000003 |
|Márcio Miguel Bittar            |2021|436020.2299999999 |
|Fernando Affonso Collor de Mello|2018|435944.08         |
|Omar José Abdel Aziz            |2017|423114.0         

# Found the company that have more revenue for the senators

In [9]:
companies = benefits.select("CNPJ_CPF", "VALOR_REEMBOLSADO", "TIPO_DESPESA", "ANO", "MES", "COD_DOCUMENTO")
companies = companies.withColumn("CNPJ_CPF", f.when(f.isnan(f.col("CNPJ_CPF")), "SEM REGISTRO").otherwise(f.col("CNPJ_CPF")))

In [10]:
benefits.select("*").where("VALOR_REEMBOLSADO = 2179546.0").show(truncate=False, vertical=True)

(0 rows)



In [11]:
companies.show()

+------------------+-----------------+--------------------+----+---+-------------+
|          CNPJ_CPF|VALOR_REEMBOLSADO|        TIPO_DESPESA| ANO|MES|COD_DOCUMENTO|
+------------------+-----------------+--------------------+----+---+-------------+
|05.914.650/0001-66|           214.79|Aluguel de imóvei...|2013|  1|       705442|
|05.914.650/0001-66|            57.34|Aluguel de imóvei...|2013|  1|       705443|
|    004.948.028-63|           5000.0|Aluguel de imóvei...|2013|  1|       682654|
|76.535.764/0001-43|            398.1|Aluguel de imóvei...|2013|  1|       705441|
|84.707.538/0001-20|           1128.0|Locomoção, hosped...|2013|  1|       682650|
|02.012.862/0001-60|          1218.15|Passagens aéreas,...|2013|  1|       682648|
|02.012.862/0001-60|           548.13|Passagens aéreas,...|2013|  1|       682649|
|02.012.862/0001-60|           1935.0|Passagens aéreas,...|2013|  1|       682651|
|02.012.862/0001-60|          1459.13|Passagens aéreas,...|2013|  1|       682652|
|02.

In [12]:
companies_more_revenue_senators_and_service = companies.select("CNPJ_CPF", "VALOR_REEMBOLSADO", "TIPO_DESPESA", "ANO")\
                    .groupBy("CNPJ_CPF", "TIPO_DESPESA", "ANO")\
                    .agg(f.max("VALOR_REEMBOLSADO").alias("VALOR_REEMBOLSADO"))\
                    .orderBy("VALOR_REEMBOLSADO", ascending=False).show(truncate=False)

+------------------+--------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+
|CNPJ_CPF          |TIPO_DESPESA                                                                                                                          |ANO |VALOR_REEMBOLSADO|
+------------------+--------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+
|29.337.655/0001-31|Divulgação da atividade parlamentar                                                                                                   |2017|270000.0         |
|04.953.008/0001-23|Divulgação da atividade parlamentar                                                                                                   |2016|256980.0         |
|38.006.631/0001-90|Divulgação da atividade parlamentar                                                  

In [13]:
companies_more_revenue_senators_and_service = companies.select("CNPJ_CPF", "VALOR_REEMBOLSADO", "TIPO_DESPESA")\
                    .groupBy("CNPJ_CPF", "TIPO_DESPESA")\
                    .agg(f.sum("VALOR_REEMBOLSADO").alias("VALOR_REEMBOLSADO"))\
                    .orderBy("VALOR_REEMBOLSADO", ascending=False).show()

+------------------+--------------------+--------------------+
|          CNPJ_CPF|        TIPO_DESPESA|   VALOR_REEMBOLSADO|
+------------------+--------------------+--------------------+
|02.012.862/0001-60|Passagens aéreas,...|1.2116719200000623E7|
|              null|Locomoção, hosped...|   6383174.409999994|
|07.575.651/0001-59|Passagens aéreas,...|   4565737.550000024|
|              null|Aluguel de imóvei...|   2993505.020000001|
|              null|Contratação de co...|   2633713.659999999|
|02.558.157/0001-62|Aluguel de imóvei...|  1957342.9699999902|
|02.575.829/0001-48|Passagens aéreas,...|  1719381.4199999876|
|22.793.293/0001-44|Contratação de co...|           1520200.0|
|              null|Divulgação da ati...|  1486625.9400000002|
|04.850.792/0001-44|Serviços de Segur...|  1441481.0100000014|
|03.434.532/0001-25|Locomoção, hosped...|           1266300.0|
|06.164.253/0001-87|Passagens aéreas,...|  1155530.4200000004|
|01.832.635/0001-18|Passagens aéreas,...|  1101089.2899

# How much employee each senators have

In [14]:
import pandas as pd

In [15]:
employee = spark.read.parquet("database/silver/remuneration")

In [16]:
senators_regex_table = benefits.select("SENADOR").drop_duplicates()

In [17]:
senatorsNameList = list(senators_regex_table.toPandas()["SENADOR"].dropna())
senatorsNameRegex = "|".join(senatorsNameList)

In [18]:
employee_liquid = employee.select("VINCULO", "CATEGORIA", "CARGO", "REFERENCIA_CARGO", "SIMBOLO_FUNCAO", "ANO_EXERCICIO", "LOTACAO_EXERCICIO", "TIPO_FOLHA", "REM_LIQUIDA", "AUXILIOS", "FILE_YEAR", "ANO_EXERCICIO")

In [19]:
employee_liquid = employee_liquid.withColumn("TOTAL", f.col("REM_LIQUIDA") + f.col("AUXILIOS"))
employee_liquid = employee_liquid.withColumn("SENADOR", f.upper("LOTACAO_EXERCICIO"))
employee_liquid = employee_liquid.withColumn("SENADOR", f.regexp_extract("SENADOR", senatorsNameRegex, 0))
employee_liquid = employee_liquid.withColumn("SENADOR", f.when(f.col("SENADOR") == "", "SENATE SERVICES").otherwise(f.col("SENADOR")))
employee_liquid = employee_liquid.withColumn("CARGO", f.when(f.isnull(f.col("CARGO")), "NONEXISTENT").otherwise(f.col("CARGO")))

In [20]:
employee_liquid.select("VINCULO", "SENADOR", "TOTAL")\
                        .where("REM_LIQUIDA > 0") \
                        .groupby("VINCULO", "SENADOR")\
                        .agg(f.count("SENADOR").alias("count"))\
                        .orderBy("count", ascending=False).show()

+------------+--------------------+-----+
|     VINCULO|             SENADOR|count|
+------------+--------------------+-----+
|     EFETIVO|     SENATE SERVICES| 1798|
|COMISSIONADO|     SENATE SERVICES| 1218|
|COMISSIONADO|         KÁTIA ABREU|   83|
|COMISSIONADO|        IZALCI LUCAS|   76|
|COMISSIONADO|       LUCAS BARRETO|   68|
|COMISSIONADO|    ROGÉRIO CARVALHO|   67|
|COMISSIONADO|       NELSINHO TRAD|   65|
|COMISSIONADO|       ROBERTO ROCHA|   62|
|COMISSIONADO|           OMAR AZIZ|   61|
|COMISSIONADO|       EDUARDO GOMES|   60|
|COMISSIONADO|  RANDOLFE RODRIGUES|   59|
|COMISSIONADO|     MECIAS DE JESUS|   58|
|COMISSIONADO|       ELIZIANE GAMA|   57|
|COMISSIONADO|    ZEQUINHA MARINHO|   56|
|COMISSIONADO|LUIZ CARLOS DO CARMO|   54|
|COMISSIONADO|     ELIANE NOGUEIRA|   54|
|COMISSIONADO|      PLÍNIO VALÉRIO|   52|
|COMISSIONADO|     FERNANDO COLLOR|   51|
|COMISSIONADO|        OTTO ALENCAR|   51|
|COMISSIONADO| WELLINGTON FAGUNDES|   51|
+------------+--------------------

In [21]:
employee_liquid.select("SENADOR")\
                        .where("REM_LIQUIDA > 0")\
                        .groupby("SENADOR")\
                        .agg(f.count("SENADOR").alias("TOTAL"))\
                        .orderBy("TOTAL", ascending=False).show()

+--------------------+-----+
|             SENADOR|TOTAL|
+--------------------+-----+
|     SENATE SERVICES| 3022|
|         KÁTIA ABREU|   85|
|        IZALCI LUCAS|   82|
|    ROGÉRIO CARVALHO|   73|
|       LUCAS BARRETO|   72|
|       ROBERTO ROCHA|   71|
|       NELSINHO TRAD|   67|
|       EDUARDO GOMES|   66|
|           OMAR AZIZ|   65|
|       ELIZIANE GAMA|   62|
|  RANDOLFE RODRIGUES|   61|
|     MECIAS DE JESUS|   60|
|    ZEQUINHA MARINHO|   59|
|     ELIANE NOGUEIRA|   58|
|      PLÍNIO VALÉRIO|   58|
|LUIZ CARLOS DO CARMO|   57|
|        OTTO ALENCAR|   55|
| WELLINGTON FAGUNDES|   54|
|     FERNANDO COLLOR|   54|
|VENEZIANO VITAL D...|   53|
+--------------------+-----+
only showing top 20 rows



# How much cost the senators employee per year

In [24]:
employee_liquid.select("VINCULO", "SENADOR", "TOTAL", "FILE_YEAR")\
                        .where("REM_LIQUIDA > 0 AND SENADOR <> 'SENATE SERVICES'") \
                        .groupby("VINCULO", "SENADOR", "FILE_YEAR")\
                        .agg(f.sum("TOTAL").alias("TOTAL"))\
                        .orderBy("TOTAL", ascending=False).show()

+------------+-------------------+---------+------------------+
|     VINCULO|            SENADOR|FILE_YEAR|             TOTAL|
+------------+-------------------+---------+------------------+
|COMISSIONADO|      NELSINHO TRAD|     2022|465207.24000000034|
|COMISSIONADO|       IZALCI LUCAS|     2022| 455605.4900000002|
|COMISSIONADO|      ROBERTO ROCHA|     2022|442890.07000000024|
|COMISSIONADO|      LUCAS BARRETO|     2022| 407766.0100000003|
|COMISSIONADO|WELLINGTON FAGUNDES|     2022| 399503.8000000001|
|COMISSIONADO|      EDUARDO GOMES|     2022| 374672.4600000002|
|COMISSIONADO|    ELIANE NOGUEIRA|     2022|          369411.3|
|COMISSIONADO|        KÁTIA ABREU|     2022|         367074.37|
|COMISSIONADO|     MARCOS ROGÉRIO|     2022| 366343.5700000002|
|COMISSIONADO|     ANGELO CORONEL|     2022|363337.54000000004|
|COMISSIONADO|   ROGÉRIO CARVALHO|     2022| 361094.7800000002|
|COMISSIONADO|    MECIAS DE JESUS|     2022| 355965.6800000001|
|COMISSIONADO|    RENAN CALHEIROS|     2